# TinyImageNet


In [ ]:
import os, random, zipfile, urllib.request
import numpy as np
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, Subset
import torchvision
import torchvision.transforms as T
import matplotlib.pyplot as plt
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

1 Download TinyImageNet Subset (if not present)

In [ ]:
data_root = "./TinyImageNet"
zip_path = os.path.join(data_root, "tiny-imagenet-200.zip")
extract_path = os.path.join(data_root, "tiny-imagenet-200")

if not os.path.exists(extract_path):
    os.makedirs(data_root, exist_ok=True)
    url = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"
    print(" Downloading TinyImageNet (~250MB)...")
    urllib.request.urlretrieve(url, zip_path)
    print(" Extracting...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(data_root)
    print(" Done.")
else:
    print(" TinyImageNet already available.")


2 Strong NUI Mask Generator

In [ ]:
def generate_strong_nui_mask(h, w, strength=1.0, exponent=2.0):
    yy, xx = np.meshgrid(np.linspace(-1, 1, h), np.linspace(-1, 1, w), indexing='ij')
    angle = np.random.uniform(0, 2*np.pi)
    direction = np.cos(angle)*xx + np.sin(angle)*yy
    cx, cy = np.random.uniform(-0.5, 0.5, 2)
    r = np.sqrt((xx - cx)**2 + (yy - cy)**2)
    radial = 1 - np.clip(r, 0, 1)**exponent
    mask = 0.6*direction + 0.4*radial
    mask = (mask - mask.min()) / (mask.max() - mask.min())
    mask = 1 + strength * (mask - 0.5)
    mask = np.clip(mask, 0.1, 1.9).astype(np.float32)
    return mask

def apply_mask_to_tensor(img_tensor, mask):
    mask = torch.tensor(mask).unsqueeze(0)
    if img_tensor.shape[1:] != mask.shape[1:]:
        mask = torch.nn.functional.interpolate(
            mask.unsqueeze(0), size=img_tensor.shape[1:], mode='bilinear', align_corners=False
        ).squeeze(0)
    return img_tensor * mask
   

3 Load TinyImageNet Subset (5 classes only, fixed labels)

In [ ]:
transform = T.Compose([
    T.Resize((64, 64)),
    T.ToTensor(),
])

dataset_path = os.path.join(extract_path, "train")
full_dataset = torchvision.datasets.ImageFolder(root=dataset_path, transform=transform)
print("Total TinyImageNet train samples:", len(full_dataset))

# Pick 8 random classes
class_indices = random.sample(range(200), 8)
print("Selected class indices:", class_indices)

#  Build class mapping (old → new)
label_map = {old: new for new, old in enumerate(class_indices)}

#  Filter dataset for only these classes
subset_samples = [
    (path, label_map[label]) for path, label in full_dataset.samples if label in class_indices
]

#  Replace samples and targets
full_dataset.samples = subset_samples
full_dataset.targets = [label for _, label in subset_samples]

# Take subset of 2000 images
subset_idx = list(range(min(2000, len(full_dataset.samples))))
dataset = Subset(full_dataset, subset_idx)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
trainset, testset = torch.utils.data.random_split(dataset, [train_size, test_size])

trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

num_classes = len(class_indices)
print(f"Training samples: {len(trainset)} | Test samples: {len(testset)} | Classes: {num_classes}")


4  Tiny CNN Model

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class SqueezeNetCustom(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        # Load pretrained SqueezeNet
        self.model = models.squeezenet1_1(pretrained=False)
        
        # Replace the final classifier
        self.model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=1)
        self.model.num_classes = num_classes

    def forward(self, x):
        return self.model(x)


5 Train & Eval Functions

In [ ]:
def train_model(model, loader, optimizer, criterion, epochs=3, apply_nui=False):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for imgs, labels in tqdm(loader, desc=f"Epoch {epoch+1}/{epochs}"):
            if apply_nui:
                imgs_aug = []
                for img in imgs:
                    if np.random.rand() < 0.7:
                        mask = generate_strong_nui_mask(64, 64,
                            strength=np.random.uniform(-3.0, 3.0),
                            exponent=np.random.uniform(0.8, 3.5)
                        )
                        img = apply_mask_to_tensor(img, mask)
                    imgs_aug.append(img)
                imgs = torch.stack(imgs_aug)
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            loss = criterion(model(imgs), labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Loss: {total_loss / len(loader):.4f}")

def evaluate(model, loader, apply_nui=False):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in loader:
            if apply_nui:
                imgs_aug = []
                for img in imgs:
                    mask = generate_strong_nui_mask(64, 64, strength=3.0, exponent=2.0)
                    img = apply_mask_to_tensor(img, mask)
                    imgs_aug.append(img)
                imgs = torch.stack(imgs_aug)
            imgs, labels = imgs.to(device), labels.to(device)
            _, predicted = model(imgs).max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return correct / total


6 Visualization Function

In [ ]:
def visualize_nui_effect(num_images=4):
    imgs, _ = next(iter(testloader))
    fig, axes = plt.subplots(num_images, 3, figsize=(9, 9))
    for i in range(num_images):
        img = imgs[i]
        mask = generate_strong_nui_mask(64, 64, strength=3.0, exponent=2.0)
        img_nui = apply_mask_to_tensor(img, mask)
        axes[i,0].imshow(img.permute(1,2,0)); axes[i,0].set_title("Original"); axes[i,0].axis("off")
        axes[i,1].imshow(mask, cmap="gray"); axes[i,1].set_title("Mask"); axes[i,1].axis("off")
        axes[i,2].imshow(img_nui.permute(1,2,0)); axes[i,2].set_title("NUI Applied"); axes[i,2].axis("off")
    plt.tight_layout()
    plt.show()

# Visualize few samples
visualize_nui_effect()

7 Baseline vs Robust Training

In [ ]:
def apply_nui_to_img(img_tensor):
    # img_tensor shape: (C, H, W)
    h, w = img_tensor.shape[1], img_tensor.shape[2]
    mask = generate_strong_nui_mask(h, w)
    return apply_mask_to_tensor(img_tensor, mask)

def train_model_mixed(model, trainloader, optimizer, criterion, epochs, nui_ratio=0.2):
    model.train()
    for epoch in range(epochs):
        for imgs, labels in trainloader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            
            # Apply NUI to a random 20% of the batch
            mask = torch.rand(len(imgs)) < nui_ratio
            imgs_aug = []
            for i, img in enumerate(imgs):
                if mask[i]:
                    imgs_aug.append(apply_nui_to_img(img))  # your NUI function
                else:
                    imgs_aug.append(img)
            imgs_aug = torch.stack(imgs_aug)

            # Forward, backward, optimize
            optimizer.zero_grad()
            outputs = model(imgs_aug)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()


In [ ]:
criterion = nn.CrossEntropyLoss()

# Baseline Model
model_clean = SqueezeNetCustom(num_classes).to(device)
optimizer = optim.Adam(model_clean.parameters(), lr=0.001)
print("\n=== Training Baseline Model (Clean) ===")
train_model(model_clean, trainloader, optimizer, criterion, epochs=3, apply_nui=False)
acc_clean = evaluate(model_clean, testloader, apply_nui=False)
acc_nui = evaluate(model_clean, testloader, apply_nui=True)
print(f"\nBefore Robust Training:\nClean: {acc_clean*100:.2f}% | NUI: {acc_nui*100:.2f}%")

# NUI-Augmented Model
model_nui = SqueezeNetCustom(num_classes).to(device)
optimizer = optim.Adam(model_nui.parameters(), lr=0.001)
print("\n=== Training Robust Model (NUI-Augmented) ===")
train_model_mixed(model_nui, trainloader, optimizer, criterion, epochs=3, nui_ratio=0.8)
acc_clean_aug = evaluate(model_nui, testloader, apply_nui=False)
acc_nui_aug = evaluate(model_nui, testloader, apply_nui=True)
print(f"\nAfter NUI-Augmented Training:\nClean: {acc_clean_aug*100:.2f}% | NUI: {acc_nui_aug*100:.2f}%")
print("-" * 40)
print(f"Accuracy Drop Before: {abs(acc_clean - acc_nui)*100:.2f}%")
print(f"Accuracy Drop After:  {abs(acc_clean_aug - acc_nui_aug)*100:.2f}%")
drop_before = abs(acc_clean - acc_nui)*100
drop_after = abs(acc_clean_aug - acc_nui_aug)*100
improvement = drop_before - drop_after
print(f"   NUI Robustness Improvement: {improvement:.2f}%")